In [18]:
import pandas as pd
import folium
from folium.plugins import HeatMapWithTime

dd=pd.read_excel('dtp_tatarstan_march2020.xls',skiprows=1,usecols='B:C')
dd['Дата ДТП']=pd.to_datetime(dd['Дата ДТП'],format='%d.%m.%Y',errors='raise')
dd['Район']=dd['Район'].str.replace('р-н','район')
dd.loc[dd['Район'].str.contains('Новосавиновский'),'Район']='Ново-Савиновский район'
locat=pd.read_csv('dd.csv',encoding='cp1251',header=None,usecols=[3])
dd['Место']=locat


dd.loc[dd['Место']=='0','Место']=dd.loc[dd['Место']=='0','Район']
dd['Место']=dd['Место'].str.lstrip(' ')
dd['Место']=dd['Место'].str.rstrip(' ')

k=0
for row in dd['Место']:
    if ('ул') in row:
        dd.loc[k,'Место']=row[2:]
    if 'пр-кт' in row:
        dd.loc[k,'Место']=row.replace('пр-кт','проспект')
    if ('Кремлевская') in row:
        dd.loc[k,'Место']=dd.loc[k,'Место'].replace('Кремлевская','Кремлёвская')
    k=k+1


import requests as r
from shapely.geometry import LineString,Point,Polygon,MultiPolygon
geom=[]


for place in dd['Место']:
    j=0
    if 'район' in place:
        d=r.get('https://nominatim.openstreetmap.org/search/'+place.split(' ')[1]+' '+place.split(' ')[0]+' Казань?format=json&polygon_geojson=1')
        f=d.json()[0]['geojson']['coordinates']
        if d.json()[0]['geojson']['type']=='MultiPolygon':
            pols=[]
            for obj in f:
                for po in obj:
                    pols.append(Polygon(po))
            geom.append(MultiPolygon(pols).centroid)
        else:
            geom.append(Polygon(d.json()[0]['geojson']['coordinates'][0]).centroid)
    else:
        d=r.get('http://nominatim.openstreetmap.org/search?',params={'format':'json','city':'Казань','street':'улица '+place,'polygon_geojson':'1'})
        for p in d.json():
            if p['geojson']['type']=='LineString':
                geom.append(LineString(p['geojson']['coordinates']).centroid)
                j=1
                break
        if j==0:
            d=r.get('http://nominatim.openstreetmap.org/search?',params={'format':'json','city':'Казань','street':place+' улица','polygon_geojson':'1'})
            for p in d.json():
                if p['geojson']['type']=='LineString':
                    geom.append(LineString(p['geojson']['coordinates']).centroid)
                    break

In [21]:
dd['geometry']=geom

days=[[] for _ in range(31)]

k=0

for date in dd['Дата ДТП']:
    days[date.day-1].append([dd.loc[k,'geometry'].y,dd.loc[k,'geometry'].x])
    k=k+1
index=[str(i+1)+' Марта' for i in range(31)]

m=folium.Map([55.8004698,49.1052443])

HeatMapWithTime(days,index).add_to(m)
m.save('accidents.html')